In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

#### Import last file updated and some adjustments

In [3]:
sae=pd.read_csv('../data/sae_dataset_updated.csv')
sae.head()

,VehicleMarketCode,VehicleNationalCode,VehicleSegmentationGlobal,VehicleModelIntlName,Bodytype_Group,Fuel_Group,Transmission_Group,Drivetype_Group,VehicleKW,VehicleDoors,...,registration_date,Mileage,Price,oricpric,PriceChangeCount,AgeOfVeh,LossVal,DevPerKm,DevPerAge,KmPerAge
0,ES,29263,Small LCV,Combo,Panel van,Diesel,Manual,2WD,73,4.0,...,2008-07-01,238000,2990,2990.0,NaN,4234.0,16331,0.068618,3.857109,56.211620
1,ES,29263,Small LCV,Combo,Panel van,Diesel,Manual,2WD,73,4.0,...,2010-06-15,90000,4499,4499.0,NaN,3520.0,14822,0.164689,4.210795,25.568182
2,ES,29263,Small LCV,Combo,Panel van,Diesel,Manual,2WD,73,4.0,...,2005-07-01,210000,2490,2700.0,1.0,5330.0,16831,0.080148,3.157786,39.399625
3,ES,30020,E,607,Sedan,Diesel,Automated,2WD,150,4.0,...,2009-10-15,259000,6500,6500.0,NaN,4745.0,47750,0.184363,10.063224,54.583772
4,ES,30020,E,607,Sedan,Diesel,Automated,2WD,150,4.0,...,2006-07-01,155000,4900,7500.0,1.0,4965.0,49350,0.318387,9.939577,31.218530


In [5]:
sae['VehicleNationalCode']=sae['VehicleNationalCode'].astype('str')
sae['VehicleGrossNewPrice']=sae['VehicleGrossNewPrice'].astype('int')
sae['registration_date']=pd.to_datetime(sae['registration_date'], format='%Y-%m-%d')
sae['SAEReporteddate']=pd.to_datetime(sae['SAEReporteddate'], format='%Y-%m-%d')
sae['VehicleProductionStart']=pd.to_datetime(sae['VehicleProductionStart'], format='%Y-%m-%d')
sae['VehicleProductionEnd']=pd.to_datetime(sae['VehicleProductionEnd'], format='%Y-%m-%d', errors='coerce')
sae2020=sae[sae.SAEReporteddate=='2020-02-03']
sae2022=sae[sae.SAEReporteddate=='2022-10-12']

In [7]:
sae.round(2).head()

,VehicleMarketCode,VehicleNationalCode,VehicleSegmentationGlobal,VehicleModelIntlName,Bodytype_Group,Fuel_Group,Transmission_Group,Drivetype_Group,VehicleKW,VehicleDoors,...,registration_date,Mileage,Price,oricpric,PriceChangeCount,AgeOfVeh,LossVal,DevPerKm,DevPerAge,KmPerAge
0,ES,29263,Small LCV,Combo,Panel van,Diesel,Manual,2WD,73,4.0,...,2008-07-01,238000,2990,2990.0,NaN,4234.0,16331,0.07,3.86,56.21
1,ES,29263,Small LCV,Combo,Panel van,Diesel,Manual,2WD,73,4.0,...,2010-06-15,90000,4499,4499.0,NaN,3520.0,14822,0.16,4.21,25.57
2,ES,29263,Small LCV,Combo,Panel van,Diesel,Manual,2WD,73,4.0,...,2005-07-01,210000,2490,2700.0,1.0,5330.0,16831,0.08,3.16,39.40
3,ES,30020,E,607,Sedan,Diesel,Automated,2WD,150,4.0,...,2009-10-15,259000,6500,6500.0,NaN,4745.0,47750,0.18,10.06,54.58
4,ES,30020,E,607,Sedan,Diesel,Automated,2WD,150,4.0,...,2006-07-01,155000,4900,7500.0,1.0,4965.0,49350,0.32,9.94,31.22
